In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import hdbscan
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np
from sklearn.metrics.cluster import contingency_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
file_path = 'diabetes_prediction_dataset.csv'  
df = pd.read_csv(file_path)
data = df.copy()
print(data.dtypes)
data=data.dropna()


numeric_columns = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
numeric_data = data[numeric_columns]

# StandardScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(numeric_data)
scaled_df = pd.DataFrame(scaled_data, columns=numeric_columns)

result_data = pd.concat([data.drop(columns=numeric_columns), scaled_df], axis=1)
print(result_data.head())

In [ ]:
num_samples_per_class = 5000

# Filter rows with diabetes value 0 and 1
diabetes_class_0 = result_data[result_data['diabetes'] == 0].sample(n=num_samples_per_class, random_state=42)
diabetes_class_1 = result_data[result_data['diabetes'] == 1].sample(n=num_samples_per_class, random_state=42)

random_sample = pd.concat([diabetes_class_0, diabetes_class_1], axis=0)
random_sample = random_sample.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
X = random_sample[['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']]
y = random_sample['diabetes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

train_data = X_train.copy()
train_data['diabetes'] = y_train.copy()

test_data = X_test.copy()
test_data['diabetes'] = y_test.copy()

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
cluster_labels = kmeans.fit_predict(X)

# Create a dictionary to store dataframes for each cluster
cluster_dataframes = {}

# Create a dataframe for each cluster and store its data points
for cluster_label in range(5):
    cluster_indices = np.where(cluster_labels == cluster_label)[0]
    cluster_dataframes[f'cluster_{cluster_label}'] = X.iloc[cluster_indices].copy()

# Access the dataframes for each cluster using the dictionary keys
for cluster_label, cluster_df in cluster_dataframes.items():
    print(f"Cluster {cluster_label} size: {len(cluster_df)}")

# Example: Access the dataframe for cluster 0
cluster_0_df = cluster_dataframes['cluster_0']

In [ ]:
# Fit KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42)
cluster_labels = kmeans.fit_predict(X)

# Combine X, y, and cluster labels into one DataFrame
data = pd.concat([pd.DataFrame(X, columns=X.columns), pd.DataFrame(cluster_labels, columns=['Clusters'])], axis=1)

plt.figure(figsize=(8, 6))
# Create a scatterplot matrix colored by cluster and with different shapes for y
g = sns.pairplot(data, hue='Clusters', palette='tab10')

# Show plot
plt.show()

In [ ]:
y_array = y.values
print(type(y_array))

In [ ]:
data = pd.concat([pd.DataFrame(X, columns=X.columns), pd.DataFrame(y_array, columns=['Class'])], axis=1)

plt.figure(figsize=(8, 6))
# Create a scatterplot matrix colored by cluster and with different shapes for y
g = sns.pairplot(data, hue='Class', palette=['#c875c4', '#005249'])

# Show plot
plt.show()